# Cable length measurement using Time Domain Reflectometry (TDR)

## Description
In this experiment, we will measure cable length by measuring time that signal needs to propagate down
the cable and back to the source. This principle is used in time-domain reflectometry, to find and locate
faults or damages on long (underground) cables. To understand why the signal would come back, and
what amount of time is needed for that, look at theoretical background of transmission lines.
Transmission line is mathematical description of a cable which takes into account its physical parameters(diameter,materials...) and describes how the cable will behave in our circuits.

![wiring diagram](img/cable_length.jpg)

## Reflection theory (short)
A signal travelling along transmission line will be partly or wholly reflected back in the opposite
direction when the signal encounters a discontinuity in the characteristic impedance of the line, or if the
far end of the line is not terminated in its characteristic impedance. Because of losses in line, reflected
pulse has smaller amplitude as incident one. The incident wave travelling down the line has not any
foreknowledge of what is at the end of the line and is only affected by the characteristic impedance Z0.
However when the pulse arrive at the open circuit at which point the current in the line is zero
(because no current flows through open circuit). The charge arrived through incident current has
nowhere to go, so by the Kirchhoff's current law there must be equal and opposite current into the end
of the line. When there is short circuit at the end of line, the voltage must be zero since there can be no
volts across a short circuit. Again, all of the energy must be reflected back up the line and the reflected
voltage must be equal and opposite to the incident voltage by Kirchhoff's voltage law: reflected and
incident voltage and current are opposite direction and equal in absolute value.

## Measuring setup
We can measure time between emitted and reflected signal. In that time, signal travels twice the cable
distance, since it travels to the end of it and back. With Red Pitaya STEMlab's sample rate(125MS/s) we can estimate cable’s length with resolution of 80 cm, additionally with x10 interpolation we can increase measuring resolution to 8 cm. Our measuring setup consists of Red Pitaya STEMlab board, a couple of T-connectors
and short(bridge) cable to connect generator output and oscilloscope input. Input has 50Ω terminator so signal
doesn’t reflect at the end of the bridge cable.

## Measuring method
Reflection is detected by locating global maximum and minimum of the incident and reflected wave. But, due to speed of the propagation of reflected wave we will not be able to distinguish reflected wave from incident one for cables shorter than ~7m. In other words, for cables shorter than 7m, signal pulse acquired on IN1 is a sum of incident and  reflected wave. To get incident and reflected wave separated we use a clever trick. When no cable is attached to the OUT1 we will generate excitation pulse. This pulse will be acquired on IN1 and saved as constant alignment signal = incident wave. Now, when we connect some cable for length measurement  in order to get reflected wave we just subtract that constant alignment signal from received signal on IN1. 


## Connection and procedure
- using bridge cable, 2x T-connector and 50 ohm termination connect in1-out1(picture above)
- run the code by clicking on play button (all cells)
- measurement is started (scope plot is running)
- connect cable for length measurement - change cable length and observe measurement

In [ ]:
import time
import numpy as np
from scipy import constants

import operator
from scipy.interpolate import interp1d

from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import HoverTool, Range1d
from bokeh.plotting import figure, output_file, show
from bokeh.models import LabelSet, Label
from bokeh.resources import INLINE 
output_notebook(resources=INLINE)

import ipywidgets as widgets

from redpitaya.overlay.mercury import mercury as overlay
fpga = overlay()

# create generator instance
gen0 = fpga.gen(0)
# create oscilloscope instance
osc0 = fpga.osc(0, 1.0)

## Global variables

In [ ]:
buffer_size = 100
dt_stop=(buffer_size)/osc0.FS
dt_step = 1/osc0.FS/10
dt = np.arange(0,dt_stop, dt_step)
vf = 0.66

## Generator configuration

In [ ]:
# arbitrary signal waveform generation (one sample pulse)
waveform = [-0.1,1, 0]
gen0.waveform      = waveform
gen0.sample_step   = 1.0
gen0.sample_offset = 0.0

gen0.burst_repetitions = 1
gen0.burst_data_len    = len(waveform)
gen0.burst_period_len  = len(waveform)
gen0.mode              = 'FINITE'

# set output amplitude, offset and enable it
gen0.amplitude     = 1.0
gen0.offset        = 0
gen0.enable        = True

## Oscilloscope configuration

In [ ]:
# data rate decimation 
osc0.decimation = 1

# trigger timing
osc0.trigger_pre  = 0
osc0.trigger_post = buffer_size

# synchronization signals from 
osc0.sync_src = fpga.sync_src['gen0']
osc0.trig_src = 0

In [ ]:
# interpolate signals to gain resolution
x     = np.linspace(1, buffer_size, buffer_size)  
x_new = np.linspace(1, buffer_size, buffer_size*10)

def get_signal_response (buffer_size):    
    # reset, start and trigger generator to get the first burst
    gen0.reset()
    gen0.start()
    gen0.trigger()
    while (osc0.status_run()): pass
    return osc0.data(buffer_size)
    
def find_peaks (signal1,signal2):
    peak1_index, peak1_value = max(enumerate(abs(signal1)), key=operator.itemgetter(1))
    peak2_index, peak2_value = max(enumerate(abs(signal2)), key=operator.itemgetter(1))
    return peak1_index, peak1_value, peak2_index, peak2_value

def calc_cable_length (vf, dt):
    peaks = find_peaks (cable_signal,alignment_signal)
    delta_sample= peaks[0]-peaks[2]
    length = round(dt * delta_sample * constants.c * vf / 2.0, 6) 
    return length

# get alignment signal - NO CABLE ATTACHED 
alignment_signal = get_signal_response (buffer_size)
alignment_signal = interp1d(x, alignment_signal, kind='cubic')(x_new)

# interpolate init cable_signal in order to get correct size(same za alignment singal) for polotting
cable_signal = np.zeros(buffer_size)
cable_signal = interp1d(x, cable_signal, kind='cubic')(x_new)

# plotting
hover = HoverTool(mode = 'vline', tooltips=[("t", "@x"), ("V", "@y")])
tools = "wheel_zoom,box_zoom,reset,pan" #crosshair
p = figure(plot_height=500, plot_width=900, title="signals", toolbar_location="right", tools=(tools, hover))
p.xaxis.axis_label='time [ns]'
p.yaxis.axis_label='voltage [V]'
p.y_range=Range1d(-0.5, 0.8)
r = [p.line(dt*1E9, cable_signal,     line_width=1, line_alpha=0.7, color ="red",legend="reflected wave"),
     p.line(dt*1E9, alignment_signal, line_width=1, line_alpha=0.7,color ="blue",legend="incident wave")]
# get and explicit handle to update the next show cell 
target = show(p,notebook_handle=True)

# define widgets labels for results displaying
w1 = widgets.Label(value='Cable length:')
w2 = widgets.Label(value="0.000")
w3 = widgets.Label(value='[m]')

w4 = widgets.Label(value='Reflected wave peak:')
w5 = widgets.Label(value="0.000")
w6 = widgets.Label(value="[V] .................@ time")
w7 = widgets.Label(value='0.000')
w8 = widgets.Label(value='[ns]')

w9  = widgets.Label(value='Incident wave peak:')
w10 = widgets.Label(value="0.000")
w11 = widgets.Label(value="[V] .................@ time")
w12 = widgets.Label(value='0.000')
w13 = widgets.Label(value='[ns]')

# define layout of label widgets
form_item_layout = widgets.Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between',
    margin = '10px',
)
# define label widgets position and combine them into box widget
form_items  = widgets.HBox([w1, w2, w3], layout=form_item_layout)
form_items1 = widgets.HBox([w4, w5, w6, w7, w8], layout=form_item_layout)
form_items2 = widgets.HBox([w9, w10, w11, w12, w13], layout=form_item_layout)

# combine HBox (horizontal) widgets to 3 VBox (verticl) ones and then all Box to one HBox 
items = [widgets.Label(str(i)) for i in range(4)]
form = widgets.HBox([widgets.VBox([form_items , form_items1, form_items2])],
    layout=widgets.Layout(
        display='flex',
        flex_flow='column',
        align_items='center',
        width='100%',
        height='140px',
        margin = '0px',
        border='2px solid orange',
    ))
form

In [ ]:
#calculating and reploting
while True:
    cable_signal = get_signal_response (buffer_size)
    cable_signal = interp1d(x, cable_signal, kind='cubic')(x_new) 
    cable_signal = cable_signal-alignment_signal
    r[0].data_source.data['y'] = cable_signal
    r[1].data_source.data['y'] = alignment_signal
    push_notebook(handle=target)
    peaks = find_peaks(cable_signal,alignment_signal)
    w2.value  = str(calc_cable_length(vf,dt_step))
    w5.value  = str(   round(peaks[1],5)     )
    w7.value  = str(dt[round(peaks[0]  )]*1E9)
    w10.value = str(   round(peaks[3],5)     )
    w12.value = str(dt[round(peaks[2]  )]*1E9)
    